请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [ ]:
# 解压数据集
%cd
%cd data/data10954
!unzip -o cat_12_test.zip
!unzip -o cat_12_train.zip

In [ ]:
# 安装相关库以及数据预处理
import paddle
import os
import numpy as np
import pandas as pd
import shutil # 文件文档处理库
import cv2    
import imghdr # 检测图片类型
from PIL import Image 
from matplotlib import pyplot as plt
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # 设置环境变量
                
%cd
train_df = pd.read_csv('data/data10954/train_list.txt', header=None, sep='\t')
train_df.columns = ['name', 'cls'] # 返回列索引列表
train_df['name'] = train_df['name'].apply(lambda x: str(x).strip().split('/')[-1]) # 切分文件名，舍去cat_12_train/
train_df['cls'] = train_df['cls'].apply(lambda x: '%02d' % int(str(x).strip())) # 使图片标签类别变成2位数字

# 将P、RGBA、L模式的图片转换为RGB模式
for i in range(len(train_df)):
    img_path = os.path.join('data/data10954/cat_12_train', train_df.at[i, 'name']) # i 元素在列中的位置 ，name 列名
    if os.path.exists(img_path) and imghdr.what(img_path): # 检测路径文件是否存在及判断类别
        img = Image.open(img_path) # 打开文件
        if img.mode != 'RGB':
            img = Image.open(img_path)
            print(img_path)
            print(img.mode)
            img = img.convert('RGB') # 转换成rgb形式
            img.save(img_path) # 保存

for img_path in os.listdir('data/data10954/cat_12_test'):
    src = os.path.join('data/data10954/cat_12_test',img_path)
    img = Image.open(src)
    if img.mode != 'RGB':
        print(img_path)
        img = img.convert('RGB') 
        img.save(src)

/home/aistudio
data/data10954/cat_12_train/tO6cKGH8uPEayzmeZJ51Fdr2Tx3fBYSn.jpg
P
data/data10954/cat_12_train/ulFBEZNRQrxn57voHAJ4UG6Mct2sw1Cj.jpg
RGBA
data/data10954/cat_12_train/F3VnNwb2K9tgMWLodrXl1f6PIEjYqhy8.jpg
L
data/data10954/cat_12_train/YfsxcFB9D3LvkdQyiXlqnNZ4STwope2r.jpg
P
data/data10954/cat_12_train/6yYs4rvFLkQJlRxdhNfMOW52EAbgHejC.jpg
RGBA
data/data10954/cat_12_train/5nKsehtjrXCZqbAcSW13gxB8E6z2Luy7.jpg
P
data/data10954/cat_12_train/yGcJHV8Uuft6grFs7QWnK5CTAZvYzdDO.jpg
P
data/data10954/cat_12_train/YGyx4qCdOb7j8tzBuNfoFHLi6gU0SE3T.jpg
RGBA
data/data10954/cat_12_train/3yMZzWekKmuoGOF60ICQxldhBEc9Ra15.jpg
P
Qt29gPjYZwv3B6RJh5yiTWXrVImue1FH.jpg


In [ ]:
# 分割数据集，并生成train_list.txt和val_list.txt
%cd 
%cd data/data10954
data = []
data_val = []
with open('train_list.txt', 'r') as f:
    n = 0
    for line in f.readlines():
        n += 1
        line = line.strip()
        if line != '':
            img_path,label = line.split('\t')
            path0 = 'cat_12_train/'+img_path[13:-4]
            if n % 10 == 0: # 1/10为验证集
                data_val.append(line+'\n')
            else:
                data.append(path0+'.jpg'+'\t'+label+'\n')           
        
# 更新train_list,并生成val_list
with open('train_list.txt', 'w') as f:
    for i in data:
        f.write(i)
with open('val_list.txt', 'w') as f:
    for i in data_val:
        f.write(i)

/home/aistudio
/home/aistudio/data/data10954


In [ ]:
# 数据增强
!pip install paddlex
import paddlex as pdx
from paddlex import transforms as T 
train_transforms = T.Compose([
    T.MixupImage(
        alpha=1.5,
        beta=1.5,
        mixup_epoch=int(300 * 25. / 27)),
    T.Resize(
        target_size=420,
        interp='CUBIC'),
    # 以图像中心点扩散裁剪长宽为目标尺寸的正方形
    T.RandomCrop(360),
    # 以一定的概率对图像进行随机水平翻转
    T.RandomHorizontalFlip(0.5),
    # 以一定的概率对图像进行随机像素内容变换，可包括亮度、对比度、饱和度、色相角度、通道顺序的调整，模型训练时的数据增强操作
    T.RandomDistort(
        brightness_range=0.25,
        brightness_prob=0.5,
        contrast_range=0.25,
        contrast_prob=0.5,
        saturation_range=0.25,
        saturation_prob=0.5,
        hue_range=18.0,
        hue_prob=0.5),
    # 以一定的概率对图像进行高斯模糊
    T.RandomBlur(0.1),
    # 对图像进行标准化
    T.Normalize([0.4848, 0.4435, 0.4023], [0.2744, 0.2688, 0.2757])
])
# 验证集增强
eval_transforms = T.Compose([
    T.Resize(
        target_size=420,
        interp='AREA'),
    T.CenterCrop(360),
    T.Normalize([0.4848, 0.4435, 0.4023], [0.2744, 0.2688, 0.2757])
])

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 KB 1.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 3.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 KB 4.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 KB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lap: filename=lap-0.4.0-cp37-cp37m-linux_x86_64.whl size=1593875 sha256=f1879dbf4e90b7ee442a0bbdef9d356c15b00adc399730b7ed5e4998588

2022-06-02 10:34:53,471-WARNING: type object 'QuantizationTransformPass' has no attribute '_supported_quantizable_op_type'
2022-06-02 10:34:53,472-WARNING: If you want to use training-aware and post-training quantization, please use Paddle >= 1.8.4 or develop version


In [ ]:
# 生成labels.txt
with open('labels.txt','w+') as f:
    for i in range(12):
        f.write(str(i)+'\t'+str(i)+'\n')
# 数据结构准备
%cd
train_dataset = pdx.datasets.ImageNet(
    data_dir='data/data10954',
    file_list='data/data10954/train_list.txt',
    label_list='data/data10954/labels.txt',
    transforms=train_transforms,
    shuffle=True) # 是否需要对数据集中样本打乱顺序
eval_dataset = pdx.datasets.ImageNet(
    data_dir='data/data10954',
    file_list='data/data10954/val_list.txt',
    label_list='data/data10954/labels.txt',
    transforms=eval_transforms)

/home/aistudio
2022-06-02 10:34:56 [INFO]	Starting to read file list from dataset...
2022-06-02 10:34:56 [INFO]	2160 samples in file data/data10954/train_list.txt
2022-06-02 10:34:56 [INFO]	Starting to read file list from dataset...
2022-06-02 10:34:56 [INFO]	2160 samples in file data/data10954/val_list.txt


In [ ]:
# 初始化模型
model = pdx.cls.ResNet50_vd_ssld(
    num_classes=len(train_dataset.labels)
)
# model = pdx.cls.ResNet101_vd_ssld(
#     num_classes=len(train_dataset.labels)
# )
# model = pdx.cls.Xception71(
#     num_classes=len(train_dataset.labels)
# )
# 开始训练
model.train(
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    num_epochs=420, #训练轮数
    train_batch_size=110, #一个step所用到的样本量
    warmup_steps=(len(train_dataset.file_list) // 110) * 6, #学习率从0经过steps轮迭代增长到设定的学习率
    learning_rate=0.025, # 学习率
    lr_decay_epochs=[40, 65, 115, 160, 205], #表示学习率在第几个epoch时衰减一次
    lr_decay_gamma=0.1, # 学习率衰减率
    save_interval_epochs=5, # 每几轮保存一次
    log_interval_steps=(len(train_dataset.file_list) // 110) * 7, # 训练日志输出间隔
    pretrain_weights='IMAGENET',# 自动下载在ImageNet图片数据上预训练的模型权重
    # pretrain_weights='output/ResNet101_vd_ssld_2/epoch_208/model.pdparams',
    save_dir='output/ResNet50_vd_ssld', # 储存模型参数
    use_vdl=False)

2022-06-02 10:35:32 [INFO]	Downloading ResNet50_vd_ssld_pretrained.pdparams from https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/legendary_models/ResNet50_vd_ssld_pretrained.pdparams


100%|██████████| 151509/151509 [00:07<00:00, 19079.02KB/s]


2022-06-02 10:35:40 [INFO]	Loading pretrained model from output/ResNet50_vd_ssld/pretrain/ResNet50_vd_ssld_pretrained.pdparams
2022-06-02 10:35:41 [WARNING]	[SKIP] Shape of pretrained params fc.weight doesn't match.(Pretrained: (2048, 1000), Actual: [2048, 12])
2022-06-02 10:35:41 [WARNING]	[SKIP] Shape of pretrained params fc.bias doesn't match.(Pretrained: (1000,), Actual: [12])
2022-06-02 10:35:41 [INFO]	There are 275/277 variables loaded into ResNet50_vd_ssld.
2022-06-02 10:36:05 [INFO]	[TRAIN] Epoch 1 finished, loss=2.4348383, acc1=0.1875598, acc5=0.57177037 .
2022-06-02 10:36:30 [INFO]	[TRAIN] Epoch 2 finished, loss=1.9166533, acc1=0.53636366, acc5=0.9052631 .
2022-06-02 10:36:54 [INFO]	[TRAIN] Epoch 3 finished, loss=0.9474112, acc1=0.72440195, acc5=0.9727273 .
2022-06-02 10:37:19 [INFO]	[TRAIN] Epoch 4 finished, loss=0.55595964, acc1=0.82057416, acc5=0.98612446 .
2022-06-02 10:37:46 [INFO]	[TRAIN] Epoch 5 finished, loss=0.43486083, acc1=0.8483253, acc5=0.98516744 .
2022-06-02 10

Exception in thread Thread-337:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 530, in _thread_loop
    batch = self._get_data()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 667, in _get_data
    batch.reraise()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/worker.py", line 169, in reraise
    raise self.exc_type(msg)
ValueError: DataLoader worker(0) caught ValueError with message:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages

KeyboardInterrupt: 

In [ ]:
!pip install paddlex
import paddlex as pdx
import glob
model = pdx.load_model('output/ResNet50_vd_ssld/epoch_275') # 加载模型
test_list = glob.glob('data/data10954/cat_12_test/*.jpg')
test_df = pd.DataFrame() # 创建表结构
for i in range(len(test_list)):
    img = Image.open(test_list[i]).convert('RGB')
    img = np.asarray(img, dtype='float32') # 转换数据类型
    result = model.predict(img[:, :, [2, 1, 0]]) # 预测结果
    test_df.at[i, 'name'] = str(test_list[i]).split('/')[-1] # 文件名
    test_df.at[i, 'cls'] = int(result[0]['category_id']) # 类别
test_df[['name']] = test_df[['name']].astype(str)
test_df[['cls']] = test_df[['cls']].astype(int)
test_df.to_csv('work/result.csv', index=False, header=False) # 生成csv文件
print('csv文件生成成功，文件目录：work/result.csv')

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
2022-06-02 12:44:48 [INFO]	Model[ResNet50_vd_ssld] loaded.
csv文件生成成功，文件目录：work/result.csv


In [2]:
# 此文档为单个模型的训练代码，我们小组尝试了ResNet50_vd_ssld、ResNet101_vd_ssld、Xception71、VGG、hrnet等多个模型
# 其中ResNet50_vd_ssld、ResNet101_vd_ssld单个模型的最高准确率都可以达到95.83%
# 最后我们通过软投票综合多个模型，可以提高准确率至97.08%
print('此文档为单个模型的训练代码，我们小组尝试了ResNet50_vd_ssld、ResNet101_vd_ssld、Xception71、VGG、hrnet等多个模型')
print('其中ResNet50_vd_ssld、ResNet101_vd_ssld单个模型的最高准确率都可以达到95.83%')
print('最后我们通过软投票综合多个模型，可以提高准确率至97.08%')

此文档为单个模型的训练代码，我们小组尝试了ResNet50_vd_ssld、ResNet101_vd_ssld、Xception71、VGG、hrnet等多个模型
其中ResNet50_vd_ssld、ResNet101_vd_ssld单个模型的最高准确率都可以达到95.83%
最后我们通过软投票综合多个模型，可以提高准确率至96.67%
